# Data Analysis

## Solutions object
Obtain the Hidden Sector from: https://zenodo.org/record/5526707#.YzwIANJBzeQ and add the missing the column `Hidden Sector` of the following data scheme

![img](./squeme.svg)

### Example of entry

In [1]:
from IPython.display import JSON 
import pandas as pd

In [2]:
JSON({'A':1})

<IPython.core.display.JSON object>

In [3]:
JSON(([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )
    )


<IPython.core.display.JSON object>

In [4]:
import pandas as pd

In [5]:
pd.DataFrame([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )

,l,k,solution,gcd,n,hidden
0,"[1, 3]",[2],"[1, 2, -3, 4, 5]",1,5,"[{'ψ': [(1, 5), (2, 4), (-3,)], 'S': 6}]"


### Read the full dataset

In [6]:
df=pd.read_json('https://zenodo.org/record/5526707/files/solutions.json?download=1')

In [7]:
FULL=True
if FULL:
    #From https://zenodo.org/record/7380817/files/solutions.json.gz?download=1
    df=pd.read_json('solutions.json.gz')
    df=df[df['solution'].str[-1].abs()<=20].reset_index(drop=True)

In [8]:
df[df['n']==5].shape

(3, 5)

In [9]:
df.shape

(96153, 5)

The `solutions` column, was generated with an ordering in absolute value, like the following

In [10]:
sorted([-2,1,-8,5],key=abs)

[1, -2, 5, -8]

This allows to take the maximum directly from the last element

In [11]:
df['solution'].str[-1]

0         3
1         6
2         4
3        20
4        12
         ..
96148    19
96149    20
96150   -20
96151   -20
96152   -20
Name: solution, Length: 96153, dtype: int64

In [12]:
df['zmax']=df['solution'].str[-1].abs()
df=df.sort_values(['n','zmax']).reset_index(drop=True)

#How many solutions with n=12 and z_max <= 20?
df[ (df['n']==12) & (df['zmax']<=20) ].shape

(65910, 6)

In [13]:
df[df['n']==5]

,l,k,solution,gcd,n,zmax
0,[-2],"[2, 3]","[1, 5, -7, -8, 9]",4,5,9
1,[3],"[-3, -2]","[2, 4, -7, -9, 10]",18,5,10
2,[-2],"[3, 1]","[1, 14, -17, -18, 20]",6,5,20


In [14]:
[1 for x in range(2)]

[1, 1]

In [15]:
df[3:4]

,l,k,solution,gcd,n,zmax
3,"[-4, -2]","[5, 2]","[1, 1, 1, -4, -4, 5]",40,6,5


In [16]:
DF = []
for i,r in df.iterrows():
    print(str(i).zfill(5),end='\r')
    repeated = set([x for x in r['solution'] if r['solution'].count(x) == 2 or r['solution'].count(x) == 3])
    for n in repeated:
        rr = r.copy()
        rr['repeated'] = [n for x in range(r['solution'].count(n))]
        rr['solution'] = [x for x in r['solution'] if x != n]
        DF.append(rr.to_dict())


In [17]:
df = pd.DataFrame(DF)

In [18]:
df.shape

(197969, 7)

### Add the new column with the list of hidden sectors

After refactoring...

In [19]:
import itertools
import math as mh

def remove_double_intersection_pairs(msvp,l):
    '''
    Example:
        msvp = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        # Remove (1,6) because is fully contained 
        # in the intersections with (_1_,-8) and (_6_,-13)
        returns: 
        [(3,4),(6,-13),(-7,14),(1,-8)]    
    '''
    for p in msvp.copy():
        tmp=msvp.copy()
        tmp.remove(p)
        fltmp=set([x for sublist in tmp for x in sublist]) 
        if l.count(p[0])==1 and l.count(p[1])==1 and set(p).issubset(fltmp):
            msvp=tmp.copy()
    return msvp
assert len(remove_double_intersection_pairs([(-10, 11), (-12, 13), (11, -12)],
                                             [-10, 11, -12, 13]))==2
assert len(remove_double_intersection_pairs([(4,4),(4,-12)],[4,-12]))==1

def get_real_massless(l,msvp):
    '''
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        msvp = [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        [] # lp    
    '''
    lp=l.copy()
    for x,y in msvp:
        #print(lp,msvp,x,y)
        if lp and x in lp and y in lp:
            minxy=min(lp.count(x),lp.count(y))
            if x!=y:
                [(lp.remove(x),lp.remove(y)) for ii in range(minxy)]
            else:
                [ lp.remove(x) for ii in range(lp.count(x)) ] # abs(2*x)==s
        else: # not lp
            break
    return lp

def get_hidden_fermions(l,s,ps=[]):
    '''
    For a given scalar Abelian charge `s`, build the 
    fermion non-zero determinant mass matrix from their
    Abelian charges in `l`, and returns the input `s` and 
    the non-zero mass entries.
    Returns an empty dictionary otherwise.
    
    The list of combinations on pairs of `l` can be given in `ps`,
    otherwise it is internally calculated.
    
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        s = 7
        ps = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        # If necessary temove [1,6] because have is fully contained 
        # in the intersections with others
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        {'s':7, '\psi':[(3,4),(6,-13),(1,6),(-7,14),(1,-8)]}

    '''
    sltn={}
    if not ps:
        ps = list(set(itertools.combinations_with_replacement(l,2)))
    msv = [p for p in ps if abs(sum(p))==s]
    lmsv=[x for sublist in msv for x in sublist]
    massives=set(lmsv)        
    massless=set(l).difference(massives)
    if not massless:
        # ========= Quality check =======================
        # Check that all the elements of l can be removed
        for INTERSECTIONS in [False,True]:
            msvp=msv.copy()
            lp=l.copy()
            if INTERSECTIONS:
                msvp = remove_double_intersection_pairs(msvp,l)
            for msvp in itertools.permutations(msvp.copy()):
                msvp=list(msvp)
                lp=get_real_massless(l,msvp)
                if not lp:
                    break # found real massless
            if not lp: # double break for INTERSECTIONS
                break # found real massless
                    
        
        if not lp:
            sltn['S']=s
            sltn['ψ']=msv
        return sltn
assert get_hidden_fermions([2, 4, 4, 5, 5, 7],9).get('S')==9    


def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        sltn=get_hidden_fermions(l,s,ps)
        if sltn:
            sltns.append(sltn)
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1 # Ana test
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8
assert get_hidden_sector([3, 3, -4, -4, -4, 5, -6, 7, -8, 9, 10, -11])[0].get('S')==1

In [20]:
import itertools
import math as mh

assert get_hidden_fermions([2, 4, 4, 5, 5, 7],9).get('S')==9    

def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        sltn=get_hidden_fermions(l,s,ps)
        if sltn:
            sltns.append(sltn)
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1 # Ana test
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8
assert get_hidden_sector([3, 3, -4, -4, -4, 5, -6, 7, -8, 9, 10, -11])[0].get('S')==1

In [21]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

CPU times: user 47.7 s, sys: 2.81 ms, total: 47.7 s
Wall time: 47.7 s


In [22]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [23]:
hs.shape #9466 solutions with hidden sector

(1028, 8)

In [24]:
hs.zmax.max()

20

In [25]:
hs[:1]

,l,k,solution,gcd,n,zmax,repeated,hidden
0,"[2, 3]","[-1, -3]","[1, -2, -3, -6]",1,6,6,"[5, 5]","[{'S': 5, 'ψ': [(-2, -3), (1, -6)]}]"


## Ask question to the solutions object

* Test the solution
* Which is the minimum hidden sector and with the least charge in absolute value
* Check for multigenerational solution
* Check for unconditional stability
* Solutions with active symmetry
* How many solutions with n=12 and zmax<=20
* Check for Majorana and Dirac fermions

Which `n` are allowed? 

In [26]:
hs['n'].unique()

array([ 6,  7,  8,  9, 10, 11, 12])

In [27]:
pd.set_option('display.max_colwidth',200)

What is the minimum n?

In [28]:
hs['n'].min()

6

What is the minimum `z_max`?

In [29]:
hs['zmax'].min()

5

In [30]:
hs[hs['zmax']==hs['zmax'].min()]

,l,k,solution,gcd,n,zmax,repeated,hidden
49,"[-1, 1, 2, 1]","[2, 0, -1, 0]","[1, 1, -2, -2, -2, -2, 3, -5]",2,10,5,"[4, 4]","[{'S': 4, 'ψ': [(-2, -2), (1, -5), (1, 3)]}]"
307,"[2, 0, 1, 0]","[-1, 1, -1, 1, -1]","[3, 3, -4, -4, -4, -4, -4, 5, 5]",2,11,5,"[2, 2]","[{'S': 8, 'ψ': [(-4, -4), (3, 5)]}]"
380,"[2, -2, 1, 2, 1]","[2, 0, 1, -1, 0]","[1, 2, 2, 2, 2, -3, -4, -4, -4, -4]",4,12,5,"[5, 5]","[{'S': 2, 'ψ': [(1, 1), (1, -3), (2, -4)]}]"


How many solutions have several dark sectors?

In [31]:
hs[hs['hidden'].apply(len)>1].shape

(0, 8)

In [32]:
hs[hs['hidden'].apply(len)>2].shape

(0, 8)

In [33]:
hs['hidden'].str[0].str['ψ']

0                              [(-2, -3), (1, -6)]
1                            [(-3, -10), (2, -15)]
2                            [(3, -20), (-5, -12)]
3                       [(1, 7), (-4, -4), (3, 5)]
4                    [(5, 11), (-7, -9), (-8, -8)]
                           ...                    
1023     [(-18, 20), (9, -11), (14, -16), (-7, 9)]
1024    [(-16, 18), (18, -20), (-11, 13), (5, -7)]
1025                 [(-9, -9), (8, 10), (2, -20)]
1026     [(18, -20), (-11, 13), (1, 1), (-14, 16)]
1027               [(10, -18), (7, -15), (5, -13)]
Name: hidden, Length: 1028, dtype: object

What is the minimum number of DM particles?

In [34]:
hs['hidden'].str[0].str['ψ'].apply(len).min()

2

What is the minimum number of DM particles?

In [35]:
hs['hidden'].str[-1].str['ψ'].apply(len).min()

2

In [36]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==3)].shape

(246, 8)

to capture the cases with until with two dark sectors:

In [37]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==2) | 
   (hs['hidden'].str[-1].str['ψ'].apply(len)==4)]

,l,k,solution,gcd,n,zmax,repeated,hidden
0,"[2, 3]","[-1, -3]","[1, -2, -3, -6]",1,6,6,"[5, 5]","[{'S': 5, 'ψ': [(-2, -3), (1, -6)]}]"
1,"[-2, 2]","[3, 0]","[2, -3, -10, -15]",4,6,15,"[13, 13]","[{'S': 13, 'ψ': [(-3, -10), (2, -15)]}]"
2,"[5, -3]","[-3, 0]","[3, -5, -12, -20]",30,6,20,"[17, 17]","[{'S': 17, 'ψ': [(3, -20), (-5, -12)]}]"
32,"[-2, 0, 2]","[-2, 1, 0, -2]","[1, 1, -4, -5, -10, -10]",4,9,10,"[9, 9, 9]","[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]"
33,"[1, 0, -1]","[-3, 3, 0, -3]","[1, 1, -3, -8, -12, -12]",9,9,12,"[11, 11, 11]","[{'S': 11, 'ψ': [(1, -12), (-3, -8)]}]"
...,...,...,...,...,...,...,...,...
1019,"[-3, -2, 1, 4, -2]","[-4, -5, 4, 5, -4]","[12, -13, -14, -14, 15, 17, -18, -18, -19, 20]",22,12,20,"[16, 16]","[{'S': 32, 'ψ': [(-14, -18), (15, 17), (12, 20), (-13, -19)]}]"
1020,"[2, 4, 0, 2, -2]","[5, -1, 3, 5, -4]","[2, 6, -7, -7, -9, -11, 15, 15, 17, 19]",4,12,20,"[-20, -20]","[{'S': 8, 'ψ': [(-9, 17), (-11, 19), (2, 6), (-7, 15)]}]"
1023,"[3, 6, 3, 0, 3]","[-6, 1, -4, -3, 3]","[-7, 9, 9, 9, -11, -11, 14, -16, -18, 20]",54,12,20,"[1, 1]","[{'S': 2, 'ψ': [(-18, 20), (9, -11), (14, -16), (-7, 9)]}]"
1024,"[-1, -2, 4, -2, 1]","[-1, 1, 4, 6, -5]","[5, 5, -7, -7, -11, 13, -16, 18, 18, -20]",8,12,20,"[1, 1]","[{'S': 2, 'ψ': [(-16, 18), (18, -20), (-11, 13), (5, -7)]}]"


In [39]:
hs['solution'].apply(lambda L: len({}.intersection(L))>0

0                                 [1, -2, -3, -6]
1                               [2, -3, -10, -15]
2                               [3, -5, -12, -20]
3                                [1, 3, -4, 5, 7]
4                             [5, -7, -8, -9, 11]
                          ...                    
1023    [-7, 9, 9, 9, -11, -11, 14, -16, -18, 20]
1024    [5, 5, -7, -7, -11, 13, -16, 18, 18, -20]
1025          [2, 8, -9, -9, -9, -9, -9, 10, -20]
1026      [1, 1, 1, 1, -11, 13, -14, 16, 18, -20]
1027    [5, 7, 7, 7, 10, -13, -15, -15, -15, -18]
Name: solution, Length: 1028, dtype: object

What is the maximum number of DM particles?

In [ ]:
hs['hidden'].str[0].str['ψ'].apply(len).max()

What is the maximum number of DM particles?

In [ ]:
hs['hidden'].str[-1].str['ψ'].apply(len).max()

In [ ]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==8)][:1]

What is the minimum number of _independent_ DM particles?

In [ ]:
lst=[{'S': 3, 'ψ': [(-17, 20), (-23, 20), (14, -17)]}]
def intersections(lst,mask=True):
    lstc=lst.copy()
    lenmax=float('inf')
    for d in lst:
        for p in d.get('ψ'):
            dd=d.get('ψ').copy()
            dd.remove(p)
            [dd.remove(q) for q in dd.copy() if set(p).intersection(set(q))]
            if len(dd) < lenmax:
                ddd=dd.copy()
                lenmax=len(dd)
                
            if mask and not dd:
                return True
    if mask:        
        return False
    return ddd

assert intersections([{'S': 3, 'ψ': [(-17, 20), (-23, 20), (14, -17)]}])
assert not intersections([{'S': 3, 'ψ': [(-17, 20), (-7, 20), (14, -17), (2,5)]}])

One excluded:

In [ ]:
intersections([{'S': 3, 'ψ': [(-17, 20), (-23, 20), (14, -17)]}])

In [ ]:
hs[hs['hidden'].apply(intersections)]

The minimum is...

In [ ]:
hs['~intersections']=hs['hidden'].apply(lambda lst: intersections(lst,mask=False))
hs[hs['~intersections'].apply(len)==hs['~intersections'].apply(len).min()].sort_values('n')

### Appendix: Same questions for Standard Model (SM) solutions
Which are solutions with three repeated integers to be asigned to SM particles

In [ ]:
df3=df[df['solution'].apply(lambda l: 3 in [l.count(x) for x in l])].reset_index(drop=True)

Remove the three repeated integers from the new `hidden` column

In [ ]:
def get_dark(l,idx=0):
    #TODO,
    xxx=list(set([x for x in l if l.count(x)==3]))
    if len(xxx)>=idx+1:
        xx=xxx[idx]
        return [x for x in l if x!=xx]
    else:
        return []
assert get_dark([1,1,2,2,2,3] )==[1, 1, 3]
assert get_dark([2,2,2,2,-5,-5,-5,7])==[2, 2, 2, 2, 7]

In [ ]:
df3['hidden']=df3['solution'].apply(get_dark).apply(get_hidden_sector)

In [ ]:
hs3=df3[df3['hidden'].apply(len)>0].reset_index(drop=True)
hs3.shape

In [ ]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=1)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

In [ ]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=2)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

In [ ]:
hs3['zmax'].min()

In [ ]:
hs3[:1]

In [ ]:
hs3[hs3['hidden'].apply(len)>1].shape

In [ ]:
hs3[hs3['hidden'].apply(len)>2].shape

In [ ]:
hs3[hs3['hidden'].apply(len)>3].shape

In [ ]:
hs3['hidden'].str[0].str['ψ'].apply(len).min()

In [ ]:
hs3[hs3['hidden'].str[0].str['ψ'].apply(len)==hs3['hidden'].str[0].str['ψ'].apply(len).min()]

Check if `sum([L,L,L,x,y])` where `(x,y) = ±S`  

In [ ]:
def check_L(r):
    L=[x for x in r.get('solution') if r.get('solution').count(x)==3 ]
    for LL in L[::3]:
        z=r.get('hidden')[0]
        for p in z.get('ψ'):
            if 3*LL+sum(list(p))==0:
                return list(p)
    return []
assert check_L({'solution':[4,4,4,5,-8,-10,13,-9, -9, -9, 15, 15, 15],
         'hidden':[{'S': 18, 'ψ': [(-9, -9), (-8, -10), (5, 13)]}]})==[]

In [ ]:
hs3[hs3.apply(check_L,axis='columns').apply(len)>0].reset_index(drop=True)

### Extend the data scheme with number of generations

In [ ]:
def get_Dirac_and_Majorana_generations(l,h):
    g=[]
    dgmax=0
    mgmax=0
    for idx in range(len(h)):
        for x,y in h[idx].get('ψ'):
            if x!=y and l.count(x)>dgmax and l.count(y)==l.count(x):
                    dgmax=l.count(x)
            if x==y and l.count(x)>mgmax:
                    mgmax=l.count(x)
        g.append({'DG':dgmax,'MG':mgmax})
    return g
l=[2, 2, -3, 15, 15, 20]
h=[{'S': 17, 'ψ': [(2, 15), (20, -3)]}]
assert get_Dirac_and_Majorana_generations(l,h)==[{'DG': 2, 'MG': 0}]

In [ ]:
hs[:3].apply(lambda d: d['l']+d['k'],axis='columns')

In [ ]:
hs['Generations']=hs.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [ ]:
hs3['Generations']=hs3.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [ ]:
hs['Gmax']=hs['Generations'].str[0].apply(lambda d: sum(d.values()))

In [ ]:
hs=hs.sort_values(['Gmax'],ascending=False).reset_index(drop=True)

In [ ]:
hs[hs['n']==8]

In [ ]:
hs[hs['n']==8][:6]

In [ ]:
hs[hs['n']==9][:3]

In [ ]:
hs[hs['n']==11][:4]

In [ ]:
hs[hs['n']==12][:4]

In [ ]:
hs3['Gmax']=-hs3['Generations'].str[0].apply(lambda d: sum(d.values()))
hs3=hs3.sort_values(['n','Gmax']).reset_index(drop=True)

In [ ]:
hs3[hs3['n']==8][:1]

In [ ]:
hs3[hs3['n']==9][:4]

In [ ]:
hs3[hs3['n']==10][:5]

In [ ]:
hs3[hs3['n']==11][:2]

In [ ]:
hs3[hs3['n']==12][:5]

In [ ]:
hs3['~intersections']=hs3['hidden'].apply(lambda lst: intersections(lst,mask=False))
hs3[hs3['~intersections'].apply(len)==hs3['~intersections'].apply(len).min()].sort_values('n')

## Appendix: Use merge to compare two DataFrames

In [ ]:
df=pd.read_json('solutions.json.gz')

In [ ]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

In [ ]:
hs2=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()
hs2.shape

Columns can only be compared if they are simple objects, like strings:

In [ ]:
hs['ss']  = hs['solution'].apply(str)
hs2['ss'] = hs2['solution'].apply(str)

Increase the with of the columns for easier visualization.

In [ ]:
pd.set_option('display.max_colwidth',200)

Compare the larger DataFrame (with filtered columns) against the smaller. The non `on='s'` repeated columns will appears with suffixes. 

In [ ]:
tmp=hs2[['ss','hidden']].merge(hs[['ss','hidden']],on='ss',how='left',suffixes=('_1','_2'))

In [ ]:
#tmp=
tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )].reset_index(drop=True)

In [ ]:
tmp['zmax']=tmp['ss'].apply(eval).str[-1].abs()
tmp['n']=tmp['ss'].apply(eval).apply(len)

In [ ]:
tmp=tmp.sort_values(['n','zmax'])

There are plenty of new solutions with the new dataset build with the Multiprocessing algorithm in the previous Chapter

In [ ]:
tmp

In [ ]:
hs.shape

In [ ]:
tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )][:5][["ss",'hidden_1']]

In [ ]:
import itertools
import math as mh

def remove_double_intersection_pairs(msvp,l):
    '''
    Example:
        msvp = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        # Remove (1,6) because is fully contained 
        # in the intersections with (_1_,-8) and (_6_,-13)
        returns: 
        [(3,4),(6,-13),(-7,14),(1,-8)]    
    '''
    for p in msvp.copy():
        tmp=msvp.copy()
        tmp.remove(p)
        fltmp=set([x for sublist in tmp for x in sublist]) 
        if l.count(p[0])==1 and l.count(p[1])==1 and set(p).issubset(fltmp):
            msvp=tmp.copy()
    return msvp
assert len(remove_double_intersection_pairs([(-10, 11), (-12, 13), (11, -12)],
                                             [-10, 11, -12, 13]))==2
assert len(remove_double_intersection_pairs([(4,4),(4,-12)],[4,-12]))==1

def get_real_massless(l,msvp):
    '''
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        msvp = [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        [] # lp    
    '''
    lp=l.copy()
    for x,y in msvp:
        #print(lp,msvp,x,y)
        if lp and x in lp and y in lp:
            minxy=min(lp.count(x),lp.count(y))
            if x!=y:
                [(lp.remove(x),lp.remove(y)) for ii in range(minxy)]
            else:
                [ lp.remove(x) for ii in range(lp.count(x)) ] # abs(2*x)==s
        else: # not lp
            break
    return lp

def get_hidden_fermions(l,s,ps=[]):
    '''
    For a given scalar Abelian charge `s`, build the 
    fermion non-zero determinant mass matrix from their
    Abelian charges in `l`, and returns the input `s` and 
    the non-zero mass entries.
    Returns an empty dictionary otherwise.
    
    The list of combinations on pairs of `l` can be given in `ps`,
    otherwise it is internally calculated.
    
    Example:
        l = [1, 3, 4, 6, -7, -8, -13, 14]
        s = 7
        ps = [(3,4),(6,-13),(1,6),(-7,14),(1,-8)]
        # If necessary temove [1,6] because have is fully contained 
        # in the intersections with others
        [1, 3, 4, 6, -7, -8, -13, 14] → [(3,4),(6,-13),(-7,14),(1,-8)]
        [1, 6, -7, -8, -13, 14] → [(6,-13),(-7,14),(1,-8)]
        [1, -7, -8, 14] → [(-7,14),(1,-8)]
        [1, -8] → [(1,-8)]
        [] → []
        
        returns:
        {'s':7, '\psi':[(3,4),(6,-13),(1,6),(-7,14),(1,-8)]}

    '''
    sltn={}
    if not ps:
        ps = list(set(itertools.combinations_with_replacement(l,2)))
    msv = [p for p in ps if abs(sum(p))==s]
    lmsv=[x for sublist in msv for x in sublist]
    massives=set(lmsv)        
    massless=set(l).difference(massives)
    if not massless:
        sltn['S']=s
        sltn['ψ']=msv
    return sltn
assert get_hidden_fermions([2, 4, 4, 5, 5, 7],9).get('S')==9    


def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        sltn=get_hidden_fermions(l,s,ps)
        if sltn:
            sltns.append(sltn)
    return sltns



In [ ]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

In [ ]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [ ]:
hs.shape #9466 solutions with hidden sector

In [ ]:
hs.solution.str[-1].abs()

In [ ]:
hs8=hs[ (hs.n==8) & (hs.solution.str[-1].abs()<20)][['solution','hidden']]

In [ ]:
hs8['hidden']=hs8['hidden'].str[0]

In [ ]:
hs8=hs8.rename({'solution':'lista','hidden':'pares'},axis='columns')

In [ ]:
hs8

In [ ]:
hs8['pares']=hs8.pares.apply(lambda d: {'S':d.get('S'),'ψ':d.get('ψ')+[(y,x) for x,y in d.get('ψ') if x!=y]})

In [ ]:
hs8.to_json('lista_pares.json',orient='records')

In [ ]:
df=pd.read_json('lista_pares.json')

In [ ]:
df.iloc[1]

In [ ]:
import numpy as np

A) Cargue los datos disponibles en:

[https://github.com/ComputationalMethods/Evaluacion_2022-1/blob/main/data/lista_pares.json](https://raw.githubusercontent.com/ComputationalMethods/Evaluacion_2022-1/main/data/lista_pares.json)

como un DataFrame de pandas. Este contendrá dos columnas. La columna `lista`, con una lista de 8 enteros (algunos de ellos repetidos) y una columna `pares` correspondiente a un diccionario
con dos items: un entero rotulado con `'S'` y una lista de pares de enteros routulada con `'ψ'`. Note que el resultado de la suma en valor absoluto de los dos enteros en cada uno de los pares reproduce el valor del correspondiente `'S'` 

Seleccione la fila del DataFrame con el índice correspondiente a los dos últimos dígitos de su cédula y desarrolle los siguientes puntos con la correspondiente  `lista` y `pares`:
1. Construya una matrix $8\times 8$ _simétrica_ donde cada fila y la correspondiente columna correspondan a un entero de la lista (incluyendo los repetidos); de manera que en los cruces de filas y columnas definidos por los pares de enteros, se genere una entrada aleatoria mayor que cero y menor o igual que 1, con las demas entradas iguales a cero.
1. Calcule el determinante y establezca explícitamente si es diferente cero (tenga en cuenta la precisión numérica). Recuerde que si el determinante es cero, se espera que al menos uno de los autovalores sea cero
2. Organize los autovalores y autovectores de menor a mayor en valor absoluto y compruebe que las matriz de autovectores organizada da lugar a los autovalores en el orden correcto. 
3. Establezca explícitamente _cuantos_ autovalores son degenerados en valor absoluto.

```
   1 2 2 2 -3 -5 -6 7
1  0 0 0 0  0  x  0 0
2  0 x x x  0  0  x 0
2  0 x x x  0  0  x 0
2  0 x x x  0  0  x 0
-3 0 0 0 0  0  0  0 x
-5 x 0 0 0  0  0  0 0
-6 0 x x x  0  0  0 0
7  0 0 0 0  x  0  0 0
```

https://stackoverflow.com/questions/65862611/integration-of-sin-x-dx-problem-in-python-programming-implementation

In [ ]:
from scipy import integrate

B)
1. Usando su número de cédula cómo semilla, obtenga un numero entero aleatorio, $n$, entre 2 y 20.
2. Haga una estimación de la siguiente integral hasta dos cifras significativas, usando el método `quad` del módulo `integrate` de Scipy
$$
\int_0^\infty \sin(x^n)\operatorname{d}x
$$
Sugerencia. Compare con el valor obtenido desde https://www.wolframalpha.com/ o [sympy](https://stackoverflow.com/questions/65862611/integration-of-sin-x-dx-problem-in-python-programming-implementation)

In [ ]:
integrate.quad(lambda x: np.sin(x**20),0,1.5,limit=5000)

In [ ]:
A=np.random.random((8,8)).round(2)
A=(A+A.transpose())/2
A[0,0:5]=0;A[0,6:]=0;
A[1:4,0]=0; A[1:4,4:6]=0;A[1:4,7]=0
#A[1:4,0]=0; A[1:4,2:6]=0;A[1:4,7]=0
#A[1:4,0]=0; A[1:4,2:6]=0;A[1:4,7]=0
A[4,0:7]=0
A[5,1:8]=0;
A[6,0]=0;A[6,4:]=0;
A[7,0:4]=0;A[7,5:]=0;

print(A)

In [ ]:
np.linalg.det(A)

In [ ]:
np.linalg.eig(A)

In [ ]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

In [ ]:
df['solution']==